<a href="https://colab.research.google.com/github/beaten-by-the-market/dart_disclosure/blob/main/snowflake_dart_annualrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Snowflake 셋팅하기
[Snowflake Quick Start 링크](https://quickstarts.snowflake.com/)

In [1]:
!pip install --upgrade snowflake-connector-python
import snowflake.connector
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 3.9 MB/s eta 0:00:00


In [2]:
# 커넥션 셋팅
# user, password는 데이터전략팀에서 안내해준 계정으로 진행하면 되며,
# account는 'nvb92521.us-east-1'으로 작성하면 됩니다.
# DUO Mobile을 통한 2-factor 인증후 연결이 됩니다.
conn = snowflake.connector.connect(
    user='yoyogwang',
    password=userdata.get('snowflake'), # colab에서 제공하는 비번 저장기능사용
    account='nvb92521.us-east-1'
    )

In [3]:
# JJK 데이터베이스가 없으면 생성
conn.cursor().execute("CREATE DATABASE IF NOT EXISTS JJK")

# JJK 데이터베이스 이용
conn.cursor().execute("USE DATABASE JJK")

In [4]:
# 스키마가 없으면 생성
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS DART_DISCLOSURE")

# 스키마 이용
conn.cursor().execute("USE SCHEMA DART_DISCLOSURE")

In [29]:
# 커넥션 닫기
conn.close()

## OpenDart에서 사업보고서 제출 공시 불러오기
[OpenDartReader패키지 깃허브링크](https://github.com/FinanceData/OpenDartReader)

In [5]:
!pip install opendartreader
import OpenDartReader
import pandas as pd
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
from tqdm.notebook import tqdm
import random


#OpenDartReader 연결
dart = OpenDartReader(userdata.get('opendart_personal')) # colab에서 제공하는 비번 저장기능사용

### 수집조건
* 사업보고서 공시 : kind_detail = A001
* 기간 : 2020~2025
* 최종여부 : N

### 날짜리스트 생성

In [27]:
#날짜리스트 생성
#연도
yr = [i for i in range(2020, 2026)]
#월
mnth_st = [i for i in range(1, 11, 3)]
mnth_end = [i for i in range(3, 15, 3)]
#일
day_st = [1,1,1,1]
day_end = [31,30,30,31]
#월일 세트
md_st = ['-'+str(i).zfill(2)+'-'+ str(j).zfill(2) for i, j in zip(mnth_st, day_st)]
md_end = ['-'+str(i).zfill(2)+'-'+ str(j).zfill(2) for i, j in zip(mnth_end, day_end)]

#날짜세트 만들기
date_st = []
for i in yr :
    for k in md_st:
        date_st.append(str(i)+str(k))

date_end = []
for i in yr :
    for k in md_end:
        date_end.append(str(i)+str(k))

# 날짜 쌍 생성
date_pairs = list(zip(date_st, date_end))
print(date_pairs)

[('2020-01-01', '2020-03-31'), ('2020-04-01', '2020-06-30'), ('2020-07-01', '2020-09-30'), ('2020-10-01', '2020-12-31'), ('2021-01-01', '2021-03-31'), ('2021-04-01', '2021-06-30'), ('2021-07-01', '2021-09-30'), ('2021-10-01', '2021-12-31'), ('2022-01-01', '2022-03-31'), ('2022-04-01', '2022-06-30'), ('2022-07-01', '2022-09-30'), ('2022-10-01', '2022-12-31'), ('2023-01-01', '2023-03-31'), ('2023-04-01', '2023-06-30'), ('2023-07-01', '2023-09-30'), ('2023-10-01', '2023-12-31'), ('2024-01-01', '2024-03-31'), ('2024-04-01', '2024-06-30'), ('2024-07-01', '2024-09-30'), ('2024-10-01', '2024-12-31'), ('2025-01-01', '2025-03-31'), ('2025-04-01', '2025-06-30'), ('2025-07-01', '2025-09-30'), ('2025-10-01', '2025-12-31')]


### 공시 불러오기

In [26]:
# 공시 불러오기 함수
def fetch_disclosures(date_pairs, kind_detail='A001', max_retries=3, base_delay=10):
    """
    API를 통해 공시 데이터를 가져오는 함수

    Args:
        date_pairs: 시작일과 종료일 튜플의 리스트
        kind_detail: 공시 유형 코드
        max_retries: 최대 재시도 횟수
        base_delay: 기본 대기 시간(초)

    Returns:
        수집된 데이터 DataFrame
    """
    disc = pd.DataFrame()
    retry_pairs = []

    # 첫 번째 시도
    for st, end in tqdm(date_pairs, desc="API 요청 진행 중"):
        try:
            print(f"기간: {st} ~ {end} 데이터 요청 중...")
            df_loop = dart.list(kind_detail=kind_detail, start=st, end=end, final=False)

            # 결과가 비어있는지 확인
            if df_loop is None or df_loop.empty:
                print(f"데이터 없음: {st} ~ {end}")
            else:
                disc = pd.concat([disc, df_loop], ignore_index=True)
                print(f"성공: {len(df_loop)}개 데이터 추가됨")

            # API 호출 사이에 지연 추가 (10-15초 랜덤 지연)
            delay = base_delay + random.uniform(0, 5)
            time.sleep(delay)

        except Exception as e:
            print(f"오류 발생: {st} ~ {end}, 원인: {str(e)}")
            retry_pairs.append((st, end, 1))  # 재시도 횟수를 1로 시작
            # 오류 발생 시 더 오래 대기 (15-20초)
            time.sleep(base_delay + 10 + random.uniform(0, 5))

    # 재시도 로직
    if retry_pairs:
        print(f"\n재시도 필요한 요청: {len(retry_pairs)}개")

        with tqdm(total=len(retry_pairs), desc="재시도 진행 중") as pbar:
            while retry_pairs:
                st, end, retry_count = retry_pairs.pop(0)

                if retry_count > max_retries:
                    print(f"최대 재시도 횟수 초과: {st} ~ {end}, 건너뜀")
                    pbar.update(1)
                    continue

                try:
                    print(f"재시도 {retry_count}/{max_retries}: {st} ~ {end}")
                    df_loop = dart.list(kind_detail=kind_detail, start=st, end=end, final=False)

                    if df_loop is None or df_loop.empty:
                        print(f"데이터 없음: {st} ~ {end}")
                    else:
                        disc = pd.concat([disc, df_loop], ignore_index=True)
                        print(f"성공: {len(df_loop)}개 데이터 추가됨")

                    pbar.update(1)

                    # 성공 후 지연 (12-18초 랜덤 지연)
                    delay = base_delay + 2 + random.uniform(0, 6)
                    time.sleep(delay)

                except Exception as e:
                    print(f"재시도 오류: {st} ~ {end}, 원인: {str(e)}")
                    # 재시도 횟수 증가
                    retry_pairs.append((st, end, retry_count + 1))
                    # 오류 후 더 긴 지연 (15-25초)
                    time.sleep(base_delay + 10 + random.uniform(0, 10))

    # 오늘 날짜 설정
    today = datetime.today().strftime('%Y%m%d')
    disc['last_update'] = today

    print(f"최종 데이터 수집 완료: {len(disc)}개 레코드")
    return disc

# 날짜 쌍 생성
date_pairs = list(zip(date_st, date_end))

# 함수 실행
disc = fetch_disclosures(date_pairs)

API 요청 진행 중:   0%|          | 0/20 [00:00<?, ?it/s]

기간: 2020-01-01 ~ 2020-03-31 데이터 요청 중...
성공: 2905개 데이터 추가됨
기간: 2020-04-01 ~ 2020-06-30 데이터 요청 중...
성공: 347개 데이터 추가됨
기간: 2020-07-01 ~ 2020-09-30 데이터 요청 중...
성공: 121개 데이터 추가됨
기간: 2020-10-01 ~ 2020-12-31 데이터 요청 중...
성공: 91개 데이터 추가됨
기간: 2021-01-01 ~ 2021-03-31 데이터 요청 중...
성공: 3372개 데이터 추가됨
기간: 2021-04-01 ~ 2021-06-30 데이터 요청 중...
성공: 460개 데이터 추가됨
기간: 2021-07-01 ~ 2021-09-30 데이터 요청 중...
성공: 172개 데이터 추가됨
기간: 2021-10-01 ~ 2021-12-31 데이터 요청 중...
성공: 88개 데이터 추가됨
기간: 2022-01-01 ~ 2022-03-31 데이터 요청 중...
성공: 3358개 데이터 추가됨
기간: 2022-04-01 ~ 2022-06-30 데이터 요청 중...
성공: 316개 데이터 추가됨
기간: 2022-07-01 ~ 2022-09-30 데이터 요청 중...
성공: 443개 데이터 추가됨
기간: 2022-10-01 ~ 2022-12-31 데이터 요청 중...
성공: 134개 데이터 추가됨
기간: 2023-01-01 ~ 2023-03-31 데이터 요청 중...
성공: 3412개 데이터 추가됨
기간: 2023-04-01 ~ 2023-06-30 데이터 요청 중...
성공: 241개 데이터 추가됨
기간: 2023-07-01 ~ 2023-09-30 데이터 요청 중...
성공: 313개 데이터 추가됨
기간: 2023-10-01 ~ 2023-12-31 데이터 요청 중...
성공: 116개 데이터 추가됨
기간: 2024-01-01 ~ 2024-03-31 데이터 요청 중...
성공: 3402개 데이터 추가됨
기간: 2024-04-01 ~ 2024-06-30 

### [첨부정정]의 rceptno 업데이트하기

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import difflib
import time
import random
from tqdm.notebook import tqdm  # Colab 환경에 최적화된 tqdm
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 효율적인 요청 세션 생성 (재시도 메커니즘 포함)
def create_session():
    session = requests.Session()
    retry_strategy = Retry(
        total=3,  # 최대 3번 재시도
        backoff_factor=0.5,  # 지수적으로 대기 시간 증가
        status_forcelist=[429, 500, 502, 503, 504],  # 재시도할 HTTP 상태 코드
        allowed_methods=["GET"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.3904.108 Safari/537.36'
    })
    return session

# 개선된 첨부정정 공시의 원본 접수번호 가져오기 함수
def get_recent(rcp_no, session=None, match=None):
    if session is None:
        session = create_session()

    try:
        r = session.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}', timeout=10)
        r.raise_for_status()  # HTTP 오류 시 예외 발생
        soup = BeautifulSoup(r.text, features="lxml")

        row_list = []
        fam = soup.find(id='family')
        if not fam:
            logger.warning(f'rcp_no={rcp_no} 첨부문서를 포함하고 있지 않습니다')
            return rcp_no  # 첨부문서가 없으면 원본 접수번호 반환

        for opt in fam.find_all('option'):
            if opt['value'] == 'null':
                continue
            title = ' '.join(opt.text.split())
            url = f'https://dart.fss.or.kr/dsaf001/main.do?{opt["value"]}'
            row_list.append([title, url])

        if not row_list:
            return rcp_no

        df = pd.DataFrame(row_list, columns=['title', 'url'])
        if match:
            df['similarity'] = df.title.apply(lambda x: difflib.SequenceMatcher(None, x, match).ratio())
            df = df.sort_values('similarity', ascending=False)

        ret_df = df[['title', 'url']]
        if ret_df.empty:
            return rcp_no

        recent_rcp_no_temp = ret_df.url[0]
        recent_rcp_no = recent_rcp_no_temp[-14:]

        return recent_rcp_no

    except Exception as e:
        logger.error(f"요청 오류 (rcp_no={rcp_no}): {e}")
        return "error"  # 오류 발생 시 특별한 값 반환

# 개선된 데이터프레임 업데이트 함수
def update_rcept_no(df):
    # 공유 세션 생성
    session = create_session()
    # 첨부정정 문서만 필터링하여 불필요한 요청 감소
    amendment_mask = df['report_nm'].str.contains('\[첨부정정\]', na=False)
    amendment_indices = df[amendment_mask].index

    # 진행 상황 표시 (Colab 환경용 tqdm)
    for index in tqdm(amendment_indices, desc="첨부정정 문서 처리 중"):
        try:
            # 랜덤한 대기 시간 적용 (1~2초)으로 요청 패턴 다양화
            delay = 1.0 + random.random()
            time.sleep(delay)

            df.at[index, 'rcept_no_new'] = get_recent(df.at[index, 'rcept_no'], session)
        except Exception as e:
            logger.error(f"처리 오류 (인덱스 {index}): {e}")
            df.at[index, 'rcept_no_new'] = "error"

    return df

# 실패한 요청 재시도 함수 (지수 백오프 적용)
def retry_failed_requests(df, max_retries=3):
    session = create_session()
    # 'error' 또는 'null' 값을 가진 행만 필터링
    error_indices = df[(df['rcept_no_new'] == 'error') | (df['rcept_no_new'] == 'null')].index

    if len(error_indices) == 0:
        return df

    logger.info(f"재시도 필요한 행: {len(error_indices)}개")

    for retry in range(max_retries):
        if len(error_indices) == 0:
            break

        logger.info(f"재시도 {retry+1}/{max_retries}: {len(error_indices)}개 행 처리 중...")

        current_indices = error_indices.copy()
        for index in tqdm(current_indices, desc=f"재시도 #{retry+1}"):
            # 지수 백오프 적용 (재시도 횟수에 따라 대기 시간 증가)
            delay = (2 ** retry) + random.random()
            time.sleep(delay)

            try:
                new_rcept_no = get_recent(df.at[index, 'rcept_no'], session)
                if new_rcept_no != "error" and new_rcept_no != "null":
                    df.at[index, 'rcept_no_new'] = new_rcept_no
                    error_indices = error_indices.drop(index)
            except Exception as e:
                logger.error(f"재시도 오류 (인덱스 {index}): {e}")

    # 최종적으로 실패한 행은 원본 접수번호 사용
    final_error_indices = df[(df['rcept_no_new'] == 'error') | (df['rcept_no_new'] == 'null')].index
    for index in final_error_indices:
        df.at[index, 'rcept_no_new'] = df.at[index, 'rcept_no']

    return df

# 메인 실행 코드
# 새 칼럼 추가
disc['rcept_no_new'] = ''

# 첨부정정 문서 접수번호 업데이트
disc = update_rcept_no(disc)

# 실패한 요청 재시도
disc = retry_failed_requests(disc)

# 빈 문자열 처리 (원본 접수번호로 대체)
disc['rcept_no_new'] = disc.apply(
    lambda row: row['rcept_no'] if row['rcept_no_new'] == '' else row['rcept_no_new'],
    axis=1
)

print(f"처리 완료: {len(disc[disc['rcept_no_new'] != disc['rcept_no']])}개 행 업데이트됨")


첨부정정 문서 처리 중:   0%|          | 0/327 [00:00<?, ?it/s]

처리 완료: 327개 행 업데이트됨


## snowflake에 업로드하기

In [30]:
try:
    # Snowflake 연결 설정
    conn = snowflake.connector.connect(
        user='yoyogwang',
        password=userdata.get('snowflake'),  # colab에서 제공하는 비번 저장기능사용
        account='nvb92521.us-east-1',
        database='JJK',
        schema='DART_DISCLOSURE'
    )

    print("Connected to Snowflake")

    # 커서 객체 생성
    cursor = conn.cursor()

    # 기존 테이블 삭제
    drop_table_query = "DROP TABLE IF EXISTS disc_list"
    cursor.execute(drop_table_query)
    print("Table 'disc_list' dropped")

    # 테이블 생성 쿼리 정의
    create_table_query = """
    CREATE TABLE disc_list (
        corp_code VARCHAR(8) COMMENT '공시대상회사의 고유번호(8자리)',
        corp_name VARCHAR(100) COMMENT '회사명',
        stock_code VARCHAR(6) COMMENT '종목코드',
        corp_cls VARCHAR(8) COMMENT '법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)',
        report_nm VARCHAR(500) COMMENT '보고서명',
        rcept_no VARCHAR(14) PRIMARY KEY COMMENT '접수번호',
        flr_nm VARCHAR(100) COMMENT '공시제출인명',
        rcept_dt VARCHAR(8) COMMENT '접수일자',
        rm VARCHAR(100) COMMENT '비고',
        last_update VARCHAR(8) COMMENT '업데이트된 날짜',
        rcept_no_new VARCHAR(14) COMMENT '접수번호_신규'
    )
    """

    # 테이블 생성
    cursor.execute(create_table_query)
    print("Table 'disc_list' created successfully")

    # 데이터 삽입을 위한 쿼리
    insert_query = """INSERT INTO disc_list (
    corp_code, corp_name, stock_code, corp_cls, report_nm,
    rcept_no, flr_nm, rcept_dt, rm, last_update, rcept_no_new)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # 데이터 프레임의 각 행을 순회하며 데이터 삽입 (tqdm으로 진행상황 표시)
    for _, row in tqdm(disc.iterrows(), total=disc.shape[0], desc="Inserting data"):
        cursor.execute(insert_query, (
            row['corp_code'], row['corp_name'], row['stock_code'], row['corp_cls'],
            row['report_nm'], row['rcept_no'], row['flr_nm'], row['rcept_dt'],
            row['rm'], row['last_update'], row['rcept_no_new']
        ))

    # 변경사항 커밋
    conn.commit()
    print("Data inserted successfully")

except snowflake.connector.errors.Error as e:
    print(f"Snowflake Error: {e}")

finally:
    if 'conn' in locals() and conn is not None:
        cursor.close()
        conn.close()
        print("Snowflake connection is closed")


Connected to Snowflake
Table 'disc_list' dropped
Table 'disc_list' created successfully


Inserting data:   0%|          | 0/20051 [00:00<?, ?it/s]

Data inserted successfully
Snowflake connection is closed
